In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt
import warnings

from data_preparation_tools import preprocessing_data
from cv_wrappers_and_tools import cv_xgb, plot_roc_auc, make_prediction_xgb, save_model_and_results

warnings.filterwarnings("ignore", category=UserWarning)
xgb.set_config(verbosity = 0)

# XGBoost

## Grid search

In [ ]:
df_train = pd.read_csv('input/cybersecurity_training.csv', sep='|', index_col=0)
df_test = pd.read_csv('input/cybersecurity_test.csv', sep='|', index_col=0)
print(df_train.shape)
print(df_test.shape)

In [ ]:
# Select features, that won't be used to create predictive models
feature_black_list = ['grandparent_category', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'score']

# Select target variable
target = 'notified'
# Prepare  training and test datasets for modeling
preprocessed_train_df, preprocessed_test_df, _, features = preprocessing_data(df_train, df_test, target, feature_black_list)

In [ ]:
trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=True, mlflow_tracking=True)
print(f'Train mean: {np.mean(trainResults)}, Test mean: {np.mean(testResults)}')

In [ ]:
# Plot to compare results of each train and validation fold after using Cross Validation
fig, ax = plt.subplots(1, 2, figsize=(16,8))
for x in range(1, 6):
    hists[['trainAUC'+'_'+str(x)]].iloc[50:].plot(ax=ax[0])
    hists[['validAUC'+'_'+str(x)]].iloc[50:].plot(ax=ax[1])
plt.show()

In [ ]:
# Trying to find the best hyperparameters using Grid Search
rang = range(19,25)
results = []
for x in rang:
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=False, mlflow_tracking=True, max_depth=x)
    print(x, np.mean(testResults))
    results.append(np.mean(testResults))

plt.plot(list(rang), results)
plt.show()

In [ ]:
rang = range(2,10)
results = []
for x in rang:
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=False, mlflow_tracking=True,
        max_depth=22, subsample = x/10)
    print(x, np.mean(testResults))
    results.append(np.mean(testResults))

plt.plot(list(rang), results)
plt.show()

In [ ]:
rang = range(2,10)
results = []
for x in rang:
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=False, mlflow_tracking=True,
        max_depth=23, subsample = 0.8, colsample_bytree = x/10)
    print(x, np.mean(testResults))
    results.append(np.mean(testResults))

plt.plot(list(rang), results)
plt.show()

In [ ]:
rang = range(0,8)
results = []
for x in rang:
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=False, mlflow_tracking=True,
        max_depth=23, subsample = 0.8, colsample_bytree = 0.5, lambdaX=x**2)
    print(x**2, np.mean(testResults))
    results.append(np.mean(testResults))

plt.plot(list(rang), results)
plt.show()

In [ ]:
rang = [0.01, 0.1, 0.5, 1, 2, 4, 6, 8, 10, 15]
results = []
for x in rang:
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=False, mlflow_tracking=True,
        max_depth=23, subsample = 0.8, colsample_bytree = 0.5, lambdaX=1, gamma=x)
    print(x, np.mean(testResults))
    results.append(np.mean(testResults))

plt.plot(list(rang), results)
plt.show()

In [ ]:
params = {
    'max_depth': 23,
    'subsample': 0.8,
    'colsample_bytree': 0.5,
    'lambdaX': 1
}
trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=True, mlflow_tracking=True, exp_name="exp_3_XGBoost" , **params)
print(f'Train mean: {np.mean(trainResults)}, Test mean: {np.mean(testResults)}')

In [ ]:
# Save results to file
modelXGB= {
    "name":"XGB-1",
    "description":"Model XGB, ze zmiennymi kategorycznymi",
    "specification": params,
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
    'features':features
}
save_model_and_results('model_xgb_1', modelXGB)

## Random Search

In [ ]:
# Trying to find the best hyperparameters using Random Search
import random
results=[]
paramList = []
for x in range(90):
    params = (random.randint(15, 30), random.uniform(0.3, 0.9), random.uniform(0.3, 0.7), random.uniform(0.1, 10), random.uniform(0, 5))
    trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features,
                                                                    mlflow_tracking=True,
                                                                    exp_name="exp_4_XGBoost",
                                                                    max_depth = params[0],
                                                                    subsample = params[1],
                                                                    colsample_bytree = params[2],
                                                                    lambdaX=params[3]**2,
                                                                    gamma=params[4])
    print(np.mean(testResults), params)
    paramList.append(params)
    # Saving results
    results.append((np.mean(testResults), params.copy()))

    # Display results of current iteration
    print(params, np.mean(testResults))

    # Every 10 iterations display top 3 results from all iterations
    if x>1 and x%10==0:
        print("\n Top 3:")
        for score, params in sorted(results, key=lambda x: x[0], reverse=True)[0:3]:
            print(params, score)
        print()

In [ ]:
# Plot the top 10 higher-scoring experiments and their hyperparameters.
maxRes = []
for i in range(1, len(paramList)):
    maxRes.append(max(results[0:i]))
imp = list(zip(results, paramList))
imp.sort(reverse=True)
for row in imp[0:10]:
    print(row)

In [ ]:
# Plot to check how many iterations in RS we need to find better score and parameters
plt.plot(list(range(len(results)-1)), maxRes)
plt.show()

In [ ]:
params = {
    'subsample': 0.9,
    'colsample_bytree': 0.5,
    'lambdax': 1,
    'max_depth':24,
    'early_stopping_rounds': 100
}

trainResults, testResults, predictions, indices, hists = cv_xgb(preprocessed_train_df, target, features, debug=True, mlflow_tracking=True, exp_name="exp_3_XGBoost", **params)
print(f'Train mean: {np.mean(trainResults)}, Valid mean: {np.mean(testResults)}')

In [ ]:
# Plot to compare results of each train and validation fold after using Cross Validation
fig, ax = plt.subplots(1, 2, figsize=(16,8))
for x in range(1, 6):
    hists[['trainAUC'+'_'+str(x)]].iloc[50:].plot(ax=ax[0])
    hists[['validAUC'+'_'+str(x)]].iloc[50:].plot(ax=ax[1])
plt.show()

In [ ]:
# Save results to file
modelXGB= {
    "name":"XGB-3",
    "description":"Model XGB, ze zmiennymi kategorycznymi",
    "specification": params,
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
    'features':features
}
save_model_and_results('model_xgb_3', modelXGB)

## Making prediction for competition

In [ ]:
params = {
    'subsample': 0.9,
    'colsample_bytree': 0.5,
    'lambdax': 1,
    'max_depth':24,
    'early_stopping_rounds': 50
}
make_prediction_xgb(train_df=preprocessed_train_df, test_df=preprocessed_test_df, features=features, target=target, test_size = 0.1, file='results_5_xgb', **params)


## Comparing the results of different predictive models

In [ ]:
with open("output/models/model_RF_1.p", "rb") as fp:
    model_RF_1 = pickle.load(fp)

with open("output/models/model_RF_2.p", "rb") as fp:
    model_RF_2 = pickle.load(fp)

# with open("output/models/model_RF_3.p", "rb") as fp:
#     model_RF_3 = pickle.load(fp)

with open("output/models/model_xgb_1.p", "rb") as fp:
    model_xgb_1 = pickle.load(fp)

with open("output/models/model_xgb_2.p", "rb") as fp:
    model_xgb_2 = pickle.load(fp)


In [ ]:
# Prepare data for the AUC-ROC curve
res2plot = []
true = preprocessed_train_df[target].sort_index()
for result in [model_RF_1, model_RF_2, model_xgb_1, model_xgb_2]:
    pred = pd.Series(sum(result["predictions"], []), index=sum(result["indices"], [])).sort_index()
    print(len(pred))
    res2plot.append((true, pred, result["name"]))

# Plot AUC-ROC curves
plot_roc_auc(res2plot)